In [ ]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/train_source')#loading the data to process which we are going to use for training 
os.mkdir('/content/drive/MyDrive/trainfiles')
os.mkdir('/content/drive/MyDrive/trainfiles/trainwav')
os.mkdir('/content/drive/MyDrive/trainfiles/trainmid')

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        y, sr = librosa.load(file)
        sr=16000
        hop_length = 512 # number of samples that we 'hopped' over
        n_fft = 2048 # window 'size'
        MFCC = librosa.feature.mfcc(y, sample_rate = sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
        #using the librosa library to create the Mel Spetrogram
        Mel = librosa.amplitude_to_db(MFCC)  # taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(Mel)[1]
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        out = '/content/drive/MyDrive/trainfiles/trainwav/wav_'+str(file_name[40:])+'.npy'
        print(np.shape(Mel))
        np.save(out,Mel)
        print('wrote file', out)
        entries1 = os.scandir('/content/drive/MyDrive/train_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used

        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The frequency has been taken such that its shape and the cqt files shape is same 
            a=np.append(np.transpose(a), np.zeros((b-np.shape(a)[1],128)), axis=0)
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/trainfiles/trainmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            print("saving", out)

  